# Data pipeline


## Step 1: Tex to pdf

1. As the very first step of the pipeline we want to inject the latex plug in the latex source file to render a pdf that has annotated regions for Proofs and Theorems

Assuming we have a folder of latex sources (here tkb-src) the function latex_sources_to_pdf actually copies the extthm file from this location to all the folders and then tries to compile a pdf file out of these sources

**It takes about 60 mins for 5720 articles to finish using 4 jobs, It might be possible that a pdf is not able to be rendered due to some bugs**

**We can't share the exact same data used for experiments, due to copyright issues but we do provide a list of arxiv ids that can be used to download and then run this script to regenerate alot of data**


In [8]:
from tex2pdf import tex2pdf

source_directory = (
    "/Users/mv96/Downloads/ACM_Multimedia/tkb-srcs"  # provide path to your arxiv folder
)
new_extthm_path = "/Users/mv96/Downloads/ACM_Multimedia/extthm.sty"  #

res = tex2pdf(
    source_directory=source_directory, new_extthm_path=new_extthm_path, n_jobs=4
).fit()


100%|█████████████████████████████████████| 5720/5720 [1:01:48<00:00,  1.54it/s]
[Parallel(n_jobs=4)]: Done 5720 out of 5720 | elapsed: 61.9min finished


In [41]:
# successful samples vs unsuccessful samples
print(len(res[0]), len(res[1]))

print("success rate:", len(res[0]) / (len(res[0]) + len(res[1])))

import random

int1 = random.randint(0, len(res[0]))
int2 = random.randint(0, len(res[1]))
print(f"some success cases :{res[0][int1]}\nsome failures:{res[1][int2][1]}")


5155 565
success rate: 0.9012237762237763
some success cases :/Users/mv96/Downloads/ACM_Multimedia/tkb-srcs/1908.04198/simplified_NPcomplete.pdf
some failures:/Users/mv96/Downloads/ACM_Multimedia/tkb-srcs/1803.01519/zk-star.tex


In [43]:
# top 5 fails
for element in res[1][:5]:
    print(element[1])


/Users/mv96/Downloads/ACM_Multimedia/tkb-srcs/1302.6695/main.tex
/Users/mv96/Downloads/ACM_Multimedia/tkb-srcs/1507.03439/weightreduction.tex
/Users/mv96/Downloads/ACM_Multimedia/tkb-srcs/1605.07503/SATsSAT_v19.tex
/Users/mv96/Downloads/ACM_Multimedia/tkb-srcs/1001.3816/main.tex
/Users/mv96/Downloads/ACM_Multimedia/tkb-srcs/1212.1941/main.tex


In [46]:
# top 5 success
for element in res[0][:5]:
    print(element)


/Users/mv96/Downloads/ACM_Multimedia/tkb-srcs/1812.02037/journal-factor.pdf
/Users/mv96/Downloads/ACM_Multimedia/tkb-srcs/1509.06361/file.pdf
/Users/mv96/Downloads/ACM_Multimedia/tkb-srcs/1806.06973/main.pdf
/Users/mv96/Downloads/ACM_Multimedia/tkb-srcs/1501.00734/arxiv.pdf
/Users/mv96/Downloads/ACM_Multimedia/tkb-srcs/1911.02675/main.pdf


### we can cleanly move pdf files to another directory

2. by providing the a target directory location (by making a new folder or pointing to already existing folder)

When we move the filtered files to a new directory we only move the file for which a **.tex** and a **.pdf** version exists, this is just to only copy the main pdf file , if there exists multiple


In [51]:
source_directory = "/Users/mv96/Downloads/ACM_Multimedia/tkb-srcs"
destination = "/Users/mv96/Downloads/ACM_Multimedia/dest-pdfs"


from copy_dest import copy_dest

copy_dest(source=source_directory, destination=destination).copy()

100%|██████████████████████████████████████| 5178/5178 [00:05<00:00, 908.75it/s]


In [54]:
# we can remove the original folder if we want
import os

!rm -r "/Users/mv96/Downloads/ACM_Multimedia/tkb-srcs"

## Step 2 A) PDF to Grobid TEI XML

1. For this step to be successful we need to already install grobid if not done so far ,
   for more info related to grobid refer the official grobid page at: https://github.com/kermitt2/grobid

in our setting we use **grobid 0.7.1** and grobid client for python https://github.com/kermitt2/grobid_client_python

Newer versions of grobid might be faster and better in terms of detection

**You might have to re run this block if it gets stuck in the first run**


In [1]:
grobid_directory = "./grobid-0.7.1"
grobid_client = "/Users/mv96/Downloads/Multimodal_proof_Extraction/Data_preprocessing/grobid_client_python"  # /Users/mv96/grobid_client_python
destination = "/Users/mv96/Downloads/Multimodal_proof_Extraction/dest-pdfs"

import os

sub_folders = [
    os.path.join(destination, element) for element in os.listdir(destination)
]  # a list grobid_valid_paths

from grobid_on_pdfs import grobid_on_pdfs

grobid_on_pdfs(
    grobid_client=grobid_client,
    grobid_directory=grobid_directory,
    sub_folders=sub_folders,
    n_jobs=4,
).fit()


100%|██████████| 1/1 [00:00<00:00, 8004.40it/s]


Starting a Gradle Daemon, 13 busy Daemons could not be reused, use --status for details
> Task :grobid-core:compileJava UP-TO-DATE
> Task :grobid-core:processResources UP-TO-DATE
> Task :grobid-core:classes UP-TO-DATE
> Task :grobid-core:jar UP-TO-DATE
> Task :grobid-trainer:compileJava UP-TO-DATE
> Task :grobid-service:compileJava UP-TO-DATE
> Task :grobid-service:processResources UP-TO-DATE
> Task :grobid-service:classes UP-TO-DATE
> Task :grobid-trainer:processResources UP-TO-DATE
> Task :grobid-trainer:classes UP-TO-DATE
> Task :grobid-trainer:jar UP-TO-DATE

> Task :grobid-service:run
14:32:03.008 [main] INFO org.grobid.service.main.GrobidServiceApplication - Found conf path: /Users/mv96/Downloads/Multimodal_proof_Extraction/Data_preprocessing/grobid-0.7.1/grobid-home/config/grobid.yaml
14:32:03.012 [main] WARN org.grobid.service.main.GrobidServiceApplication - Running with default arguments: "server" "/Users/mv96/Downloads/Multimodal_proof_Extraction/Data_preprocessing/grobid-0.7

INFO  [2023-07-07 12:32:05,259] com.hubspot.dropwizard.guicier.DropwizardModule: Added guice injected health check: org.grobid.service.resources.HealthResource
WARN  [2023-07-07 12:32:05,265] org.grobid.core.main.GrobidHomeFinder: Grobid config file location was not explicitly set via 'org.grobid.config' system variable, defaulting to: /Users/mv96/Downloads/Multimodal_proof_Extraction/Data_preprocessing/grobid-0.7.1/grobid-home/config/grobid.yaml
INFO  [2023-07-07 12:32:05,321] org.grobid.service.GrobidRestService: Initiating Servlet GrobidRestService
INFO  [2023-07-07 12:32:05,326] org.grobid.core.main.LibraryLoader: Loading external native sequence labelling library
INFO  [2023-07-07 12:32:05,331] org.grobid.core.main.LibraryLoader: Loading Wapiti native library...
INFO  [2023-07-07 12:32:05,363] org.grobid.core.main.LibraryLoader: Native library for sequence labelling loaded
INFO  [2023-07-07 12:32:05,370] org.grobid.core.lexicon.Lexicon: Initiating dictionary
INFO  [2023-07-07 12:3

100%|██████████| 1/1 [00:00<00:00, 75.68it/s]


In [10]:
import glob
from shutil import rmtree
from tqdm import tqdm

grobid_files = glob.glob("/Users/mv96/Downloads/ACM_Multimedia/dest-pdfs/**/**.tei.xml")
len(grobid_files)


381

## Visualizing Grobid output


In [ ]:
from grobid_clean import Preprocess_using_grobid
import time

tick = time.time()
prep = Preprocess_using_grobid()
final = prep.fit(
    grobid_xml="/Users/mv96/Downloads/ACM_Multimedia/dest-pdfs/1911.02675/main.tei.xml",
    show_results=True,
)
tock = time.time()
print(tock - tick)


generating visualizations
inside_visualizations


100%|███████████████████████████████████████████| 50/50 [00:13<00:00,  3.61it/s]


1 Least-Squares, Preconditioning and Randomized Newton Methods
[1, (200.0, 1419.9722222222222), (1500.0, 1554.2499999999998), 'We consider an overdetermined least-squares optimization problem min x∈R d f (x) : = 1 2 ∥Ax − b∥ 2 2 ,(1)']
[1, (200.0, 1586.861111111111), (1500.0277777777776, 1771.111111111111), 'where A ∈ R n×d is a data matrix with n ⩾ d and b ∈ R n is a vector of observations.Let x * : = A † b be the optimal least-squares solution, where A † denotes the pseudo-inverse of A. In general, computing the solution x * -by, first, forming the matrix A ⊤ A, and then, solving the linear system A ⊤ Ax = A ⊤ b through either a QR decomposition or a Cholesky factorization of A ⊤ A -takes O(nd 2 ) computational time, which can be prohibitive for a large sample size n.']
[1, (200.0, 1419.9722222222222), (1500.0, 1554.2499999999998), 'We consider an overdetermined least-squares optimization problem min x∈R d f (x) : = 1 2 ∥Ax − b∥ 2 2 ,(1)']
[1, (200.0, 1297.888888888889), (1500.027777

[3, (200.0, 621.7222222222222), (1500.0277777777776, 877.8333333333334), 'In this paper, we provide an exact analysis of the iterative Hessian sketch, where, in each step, we compute the gradient g(x t ) = A ⊤ (Ax t − b), we draw a randomized approximate Hessian H t = A ⊤ S t ⊤ S t A, and we perform the update (4).We will consider this algorithm with sketching matrices S t that are either refreshed (independently drawn at each iteration), or, fixed (S t = S 0 for all t ⩾ 0).Further, we also provide an exact analysis of the iterative Hessian sketch with momentum acceleration, which, starting from initial points x 0 , x 1 ∈ R d , at each iteration t ⩾ 1, performs the update']
[3, (569.75, 910.7222222222222), (1500.0, 967.4444444444445), 'x t+1 = x t − µ t H t −1 g(x t ) + β(x t − x t−1 ) .(5)']
[3, (200.0, 1053.9722222222222), (1500.0, 1121.9166666666667), 'Various types of randomized sketches are possible, and we describe the few of them that we consider in our algorithmic procedures.']

[5, (200.0, 1211.611111111111), (1500.0277777777776, 1543.7777777777776), 'which is dominated by the term O (nd log(1/ϵ)) for large sample sizes n ≫ d and small enough relative error ε.To our knowledge, the computational complexity (8) is the best known complexity to achieve an ε-relative error solution in highly overdetermined least-squares problems.It should be noted that the empirical sketch size prescribed by Rokhlin and Tygert is actually m pr = 4d, for which they observe an empirical performance similar to m = 4d 2 , as reported in [19].Consequently, we use in this paper m pr as the baseline empirical choice for the pre-conditioned C.G. method.In a related vein, sketching methods have also been used to restrict the optimization variable to a lower-dimensional subspace.Gower and Richtarik [7] propose a randomized iterative method with linear convergence rate, which, at each iteration, performs the proximal update']
[5, (200.0, 1544.75), (1500.0277777777776, 1920.138888888889), ', 

...

[38, (200.0, 432.52777777777777), (748.2777777777777, 462.8333333333333), 'We introduce the following function in β,']
[38, (506.6111111111111, 500.3611111111111), (1500.0, 591.4444444444443), 'γ α t (β) : = 1/λ − 1/λ + (β − x) 2t x − λ −1 + λ −1 − − x x dx .(64)']
[38, (200.0, 616.4166666666666), (1499.9722222222222, 689.0), 'The function γ α t is strongly convex, and goes to +∞ as |β| → +∞.Thus, it admits a unique minimizer β t , which is characterized by first-order optimality conditions,']
[38, (536.8888888888889, 725.5277777777777), (1179.9444444444443, 816.3333333333333), '1/λ − 1/λ + β t − x 2t−1 (x − λ −1 + )(λ −1 − − x) x dx = 0 .']
[38, (200.0, 832.1666666666666), (1500.0, 1021.1666666666666), 'We recognize the equation ( 63), and therefore, we have that β * t = β t . Setting ν = √ λ + + √ λ − √ λ + − √ λ − 2 and σ = λ −1 + + λ −1 − 2 , we get that, up to a positive constant independent of β, the function γ α t is equal to']
[38, (686.7222222222222, 1049.8333333333333), (1500

[41, (198.44444444444443, 841.1666666666666), (1500.0277777777776, 1491.638888888889), 'By assumption, |λ 1 | ⩾ |λ 2 |, |λ 3 |.Hence, we must have |λ 1 | = |λ 2 | = |λ 3 |.Since ν 2 ν 3 ̸ = 0 (otherwise, the second equation of the system (72) has a null left-hand side and a non-null right-hand side), it follows that either |λ 2 | = Λ(µ, β) and ν 2 ̸ = 0, or, |λ 3 | = Λ(µ, β) and ν 3 ̸ = 0, which concludes the proof.Lemma 15.Assume that Ax 0 ̸ = Ax * and Ax 1 = Ax * , i.e., ∆ 0 ̸ = 0 and ∆ 1 = 0.Then, one of the following statements must be true, (a) ν 1 ̸ = 0, and |λ 1 | = Λ(µ, β), (b) ν 2 ̸ = 0 and |λ 2 | = Λ(µ, β), (c) ν 3 ̸ = 0 and |λ 3 | = Λ(µ, β). then the result is proved.Hence, let us assume that ν 1 = 0.By assumption, we also have ∥∆ 1 ∥ 2 2 = 0 and ∥∆ 0 ∥ 2 2 ̸ = 0. From the linear dynamics (31) for t = 0, 1, 2, we obtain that c 0 = β 2 ∥∆ 0 ∥ 2 2 , c 1 = 0 and c 2 = β 4 ∥∆ 0 ∥ 2 2 .Further, using the formula (71) for t = 0, 1, 2, we then obtain Since β 2 ∥∆ 1 ∥ 2 2 ̸ = 0, we 

[45, (200.0, 911.361111111111), (1500.0, 983.6111111111111), 'Lemma 21.Suppose that ρ * ̸ = 1 2 .Then, for any α ≥ 0, the polynomial P α has no root within the interval (0, ρ * ).']
[45, (200.0, 1015.861111111111), (1500.0, 1434.1944444444443), 'Proof.We defer the proof to Appendix B.6.Proof.Suppose first that ρ * ̸ = 1 2 .Assume that for some parameters β ≥ 0 and α ≥ 0, the root radius Λ( α, β) satisfies Λ( α, β) < ρ * . For any value of ρ * ∈ (0, 1), it holds that inf α≥0,β≥0 Λ(α, β) = ρ * , and the infimum is uniquely attained at (α * , β * ) = (1, 0).']
[45, (200.0, 1478.2777777777776), (1243.75, 1593.2222222222224), 'Then, we must have β > 0. Indeed, we have already shown in Corollary 1 that inf α≥0 Λ(α, 0) = ρ * .']
[45, (200.0, 1626.0833333333333), (1500.0, 1735.0555555555554), 'From Lemma 20, we know that there must exist some parameters 0 < β ′ < ρ * and α ′ ≥ 0 such that χ α ′ ,β ′ (ρ * ) = 0.That is, the polynomial P α ′ has a root β ′ ∈ (0, ρ * ).But this is contradiction w

[42, (200.0, 209.66666666666666), (1411.0, 316.75), 'Case 2: two roots are equal Assume that λ 1 = λ 2 , and λ 1 ̸ = λ 3 .Then, there exist ν 1 , ν 2 , ν 3 ∈ C such that for any t ⩾ 0,']
[42, (200.0, 397.6944444444444), (1500.0277777777776, 957.75), 'Lemma 16.Assume that Ax 0 = Ax * and Ax 1 ̸ = Ax * , i.e., ∆ 0 = 0 and ∆ 1 ̸ = 0.Then, one of the following statements must be true, Proof.First, we assume that Λ(µ, β) = |λ 1 | ⩾ |λ 3 |.If ν 1 ̸ = 0 or ν 2 ̸ = 0, then the result is proved.Hence, let us assume that ν 1 = ν 2 = 0.By assumption, we also have ∥∆ 0 ∥ 2 2 = 0 and ∥∆ 1 ∥ 2 2 ̸ = 0. From the linear dynamics (31) for t = 0, 1, 2, we obtain that c 0 = 0, c 1 = β 2 ∥∆ 1 ∥ 2 2 and c 2 = β 2 η∥∆ 1 ∥ 2 2 .Using the formula (74) for t = 0, we then obtain ν 3 = 0, and hence, c t = 0 for all t ⩾ 0. But this is in contradiction with the equality c 1 = β 2 ∥∆ 1 ∥ 2 2 , which is positive.Therefore, we must have ν 1 ̸ = 0 or ν 2 ̸ = 0. Now, we assume that Λ(µ, β) = |λ 3 | > |λ 1 |.If ν 3 ̸ = 

[50, (253.86111111111111, 542.5), (1500.0, 635.3611111111111), 'Then, there exists α ∈ (0, 1) such that for any α ≥ 0, 0 < β + (α) < ρ * if and only if α ∈ (α, 1).Further, β + (α) = ρ * and β + (1) = 0.']
[50, (200.0, 642.6666666666666), (1500.0, 714.0), 'Proof.Fix α ∈ R. Using the expression of β + (α) given in Lemma 22, we have that β + (α) < ρ * if and only if']
[50, (200.0, 1020.9166666666665), (1500.0, 1089.638888888889), 'Inequality (82) can only be true for α > −1, which we assume from now on.Squaring both sides and after a few manipulations, we find that inequality (82) is equivalent to']
[50, (200.0, 1195.4722222222222), (1155.0833333333335, 1227.0277777777778), 'The polynomial Q has two distinct real roots α 1 , α 2 , which are given by']
[50, (200.0, 1370.0555555555557), (1500.0, 1437.9999999999998), ', the dominant coefficient of Q is negative, and Q takes positive values between its two roots.Therefore,']
[50, (200.0, 1519.8333333333333), (1500.0, 1589.8055555555557), ') a

## Step 2 B) PDF to PDF Alto XML

We need pdfalto information to extract the font based information

For this step we must build/install pdfalto on our system and point to the pdfalto installation in our function

refer https://github.com/kermitt2/pdfalto for more information on pdfalto

we used Pdfalto **v.0.4** in our experiments


In [19]:
from pdfalto_on_pdfs import pdfalto_on_pdfs
import os

destination = "/Users/mv96/Downloads/ACM_Multimedia/dest-pdfs"
pdfalto = "./pdfalto/pdfalto"

pdfs = pdfalto_on_pdfs(pdfalto=pdfalto, n_jobs=4, main_folder=destination).fit()


5178











  0%|                                                  | 0/5178 [00:00<?, ?it/s]








  0%|                                          | 8/5178 [00:00<06:11, 13.93it/s]Syntax Error: Couldn't read xref table
Syntax Warning: PDF file is damaged - attempting to reconstruct xref table...









  0%|                                         | 12/5178 [00:01<13:05,  6.58it/s]








  0%|▏                                        | 16/5178 [00:02<14:42,  5.85it/s]








  0%|▏                                        | 20/5178 [00:03<15:05,  5.70it/s]








  0%|▏                                        | 24/5178 [00:04<17:08,  5.01it/s]








  1%|▏                                        | 28/5178 [00:04<15:20,  5.60it/s]








  1%|▎                                        | 32/5178 [00:05<14:07,  6.07it/s]








  1%|▎                                        | 36/5178 [00:06<15:27,  5.54it/s]








  1%|▎                                        | 40/5178 [00:06<12:39,  6.










  3%|█▏                                      | 152/5178 [00:26<21:05,  3.97it/s]








  3%|█▏                                      | 156/5178 [00:27<20:04,  4.17it/s]








  3%|█▏                                      | 160/5178 [00:28<17:49,  4.69it/s]








  3%|█▎                                      | 164/5178 [00:29<17:59,  4.64it/s]








  3%|█▎                                      | 168/5178 [00:29<15:13,  5.49it/s]








  3%|█▎                                      | 172/5178 [00:29<12:41,  6.58it/s]








  3%|█▎                                      | 176/5178 [00:30<11:29,  7.26it/s]








  3%|█▍                                      | 180/5178 [00:30<10:06,  8.24it/s]








  4%|█▍                                      | 184/5178 [00:31<09:17,  8.96it/s]








  4%|█▍                                      | 188/5178 [00:31<10:23,  8.00it/s]








  4%|█▍                                      | 192/5178 [00:32<11:04,  7.51it/s]








  4%|█▌     

 13%|█████▎                                  | 684/5178 [01:52<13:26,  5.57it/s]








 13%|█████▎                                  | 688/5178 [01:53<12:05,  6.19it/s]








 13%|█████▎                                  | 692/5178 [01:53<10:25,  7.17it/s]








 13%|█████▍                                  | 696/5178 [01:54<11:25,  6.54it/s]








 14%|█████▍                                  | 700/5178 [01:55<12:05,  6.17it/s]








 14%|█████▍                                  | 704/5178 [01:55<11:13,  6.64it/s]








 14%|█████▍                                  | 708/5178 [01:56<10:04,  7.40it/s]








 14%|█████▌                                  | 712/5178 [01:56<09:15,  8.04it/s]








 14%|█████▌                                  | 716/5178 [01:57<10:02,  7.41it/s]








 14%|█████▌                                  | 720/5178 [01:58<11:54,  6.24it/s]








 14%|█████▌                                  | 724/5178 [01:58<11:46,  6.30it/s]








 14%|█████▌          

 23%|█████████                              | 1204/5178 [03:27<13:27,  4.92it/s]








 23%|█████████                              | 1208/5178 [03:27<11:53,  5.56it/s]








 23%|█████████▏                             | 1212/5178 [03:28<11:36,  5.69it/s]








 23%|█████████▏                             | 1216/5178 [03:28<09:34,  6.89it/s]








 24%|█████████▏                             | 1220/5178 [03:29<09:52,  6.68it/s]








 24%|█████████▏                             | 1224/5178 [03:30<10:47,  6.10it/s]








 24%|█████████▏                             | 1228/5178 [03:30<10:25,  6.31it/s]








 24%|█████████▎                             | 1232/5178 [03:31<10:52,  6.05it/s]








 24%|█████████▎                             | 1236/5178 [03:33<15:03,  4.36it/s]








 24%|█████████▎                             | 1240/5178 [03:33<13:35,  4.83it/s]








 24%|█████████▎                             | 1244/5178 [03:34<12:27,  5.26it/s]








 24%|█████████▍      

...

 73%|████████████████████████████▋          | 3804/5178 [11:02<04:20,  5.27it/s]








 74%|████████████████████████████▋          | 3808/5178 [11:02<03:53,  5.88it/s]








 74%|████████████████████████████▋          | 3812/5178 [11:02<03:24,  6.69it/s]








 74%|████████████████████████████▋          | 3816/5178 [11:03<03:39,  6.19it/s]








 74%|████████████████████████████▊          | 3820/5178 [11:04<03:56,  5.75it/s]








 74%|████████████████████████████▊          | 3824/5178 [11:05<03:51,  5.85it/s]








 74%|████████████████████████████▊          | 3828/5178 [11:05<03:37,  6.21it/s]








 74%|████████████████████████████▊          | 3832/5178 [11:06<03:57,  5.68it/s]








 74%|████████████████████████████▉          | 3836/5178 [11:06<03:14,  6.89it/s]








 74%|████████████████████████████▉          | 3840/5178 [11:07<03:06,  7.17it/s]








 74%|████████████████████████████▉          | 3844/5178 [11:08<03:46,  5.89it/s]








 74%|████████████████

 84%|████████████████████████████████▊      | 4352/5178 [12:43<02:46,  4.95it/s]








 84%|████████████████████████████████▊      | 4356/5178 [12:44<02:36,  5.26it/s]








 84%|████████████████████████████████▊      | 4360/5178 [12:44<02:18,  5.90it/s]








 84%|████████████████████████████████▊      | 4364/5178 [12:45<02:14,  6.05it/s]Syntax Error (179): Unknown operator 'pagesize'
Syntax Error (181): Unknown operator 'width'
Syntax Error (194): Unknown operator 'pt'
Syntax Error (196): Unknown operator 'height'
Syntax Error (198): Unknown operator 'pt'









 84%|████████████████████████████████▉      | 4368/5178 [12:45<01:58,  6.81it/s]








 84%|████████████████████████████████▉      | 4372/5178 [12:46<02:10,  6.17it/s]Syntax Error: Couldn't read xref table
Syntax Warning: PDF file is damaged - attempting to reconstruct xref table...
Syntax Error (35963): Missing 'endstream'
Syntax Error: Couldn't find trailer dictionary
Syntax Error: Couldn't read xref table









 94%|████████████████████████████████████▊  | 4880/5178 [14:15<00:38,  7.65it/s]








 94%|████████████████████████████████████▊  | 4884/5178 [14:16<00:44,  6.62it/s]








 94%|████████████████████████████████████▊  | 4888/5178 [14:17<00:56,  5.09it/s]








 94%|████████████████████████████████████▊  | 4892/5178 [14:18<00:54,  5.27it/s]








 95%|████████████████████████████████████▉  | 4896/5178 [14:18<00:52,  5.32it/s]








 95%|████████████████████████████████████▉  | 4900/5178 [14:19<00:54,  5.15it/s]








 95%|████████████████████████████████████▉  | 4904/5178 [14:20<00:53,  5.15it/s]








 95%|████████████████████████████████████▉  | 4908/5178 [14:21<00:56,  4.77it/s]








 95%|████████████████████████████████████▉  | 4912/5178 [14:21<00:49,  5.35it/s]








 95%|█████████████████████████████████████  | 4916/5178 [14:22<00:45,  5.78it/s]








 95%|█████████████████████████████████████  | 4920/5178 [14:23<00:53,  4.82it/s]








 95%|████████████████

Syntax Error: Properties 'OCPrint' is unknown
Syntax Error: Properties 'OCView' is unknown
Syntax Error: Properties 'OCPrint' is unknown
Syntax Error: Properties 'OCView' is unknown
Syntax Error: Properties 'OCPrint' is unknown
Syntax Error: Properties 'OCView' is unknown
Syntax Error: Properties 'OCPrint' is unknown
Syntax Error: Properties 'OCView' is unknown
Syntax Error: Properties 'OCPrint' is unknown
Syntax Error: Properties 'OCView' is unknown
Syntax Error: Properties 'OCPrint' is unknown
Syntax Error: Properties 'OCView' is unknown
Syntax Error: Properties 'OCPrint' is unknown
Syntax Error: Properties 'OCView' is unknown
Syntax Error: Properties 'OCPrint' is unknown
Syntax Error: Properties 'OCView' is unknown
Syntax Error: Properties 'OCPrint' is unknown
Syntax Error: Properties 'OCView' is unknown
Syntax Error: Properties 'OCPrint' is unknown
Syntax Error: Properties 'OCView' is unknown
Syntax Error: Properties 'OCPrint' is unknown
Syntax Error: Properties 'OCView' is unknown










 97%|█████████████████████████████████████▋ | 5000/5178 [14:37<00:27,  6.48it/s]








 97%|█████████████████████████████████████▋ | 5004/5178 [14:38<00:31,  5.46it/s]








 97%|█████████████████████████████████████▋ | 5008/5178 [14:39<00:32,  5.18it/s]








 97%|█████████████████████████████████████▋ | 5012/5178 [14:40<00:28,  5.89it/s]








 97%|█████████████████████████████████████▊ | 5016/5178 [14:40<00:26,  6.22it/s]








 97%|█████████████████████████████████████▊ | 5020/5178 [14:40<00:19,  8.06it/s]








 97%|█████████████████████████████████████▊ | 5024/5178 [14:41<00:17,  8.90it/s]








 97%|█████████████████████████████████████▊ | 5028/5178 [14:41<00:16,  8.95it/s]








 97%|█████████████████████████████████████▉ | 5032/5178 [14:42<00:17,  8.45it/s]








 97%|█████████████████████████████████████▉ | 5036/5178 [14:42<00:17,  8.11it/s]








 97%|█████████████████████████████████████▉ | 5040/5178 [14:43<00:17,  8.02it/s]








 97%|███████

# visualizing boxes


In [ ]:
from visualize_annot import annotations_page
# try this - /Users/mv96/Desktop/temp/1902.11202/1902.11202_annot.xml
# /Users/mv96/Desktop/temp/1910.12458/1910.12458_annot.xml
annot = annotations_page(
    labels_xml="/Users/mv96/Desktop/temp/1902.11202/1902.11202_annot.xml",
    show_images=True,
)

annotations = annot.fit()


# assigning color labels to the fonts (using latex sources) for viz

1. In this example we can see all the grobid blocks visualized with colours surrounding these paragraphs

2. These paragraphs indicate the type for ex: blue-basic, Green - Theorem


In [ ]:
# try=/Users/mv96/new/dest-pdfs/1509.06361/file.tei.xml
from labelling import assigning_labels

# name 1712.06239/main.pdf
grobid_xml = "/Users/mv96/Downloads/ACM_Multimedia/dest-pdfs/0907.1724/PlanarTutteArXiv3.tei.xml"  # grobid xml
labels_xml = "/Users/mv96/Downloads/ACM_Multimedia/dest-pdfs/0907.1724/PlanarTutteArXiv3_annot.xml"  # annotations xml

table, scales = assigning_labels(
    show_images=True, grobid_xml=grobid_xml, labels_xml=labels_xml
).fit()

table


100%|███████████████████████████████████████████| 26/26 [00:06<00:00,  3.76it/s]


1 Introduction
[2, (200.03563410896913, 457.3204026367601), (1468.539381629429, 486.4395339304793), 'The Tutte polynomial of a graph G = (V, E) (see [11,13]) is the two-variable polynomial', 'basic']
[2, (453.49745216120874, 529.0068136461069), (1475.2628015536473, 603.5273385790792), 'T (G; x, y) = A⊆E (x − 1) κ(V,A)−κ(V,E) (y − 1) |A|−n+κ(V,A) ,(1)', 'basic']
[2, (200.03563410896913, 457.3204026367601), (1468.539381629429, 486.4395339304793), 'The Tutte polynomial of a graph G = (V, E) (see [11,13]) is the two-variable polynomial', 'basic']
[2, (200.03563410896913, 302.88897922980266), (576.8527598617397, 344.98390662482615), '1 Introduction', 'basic']
[2, (200.03563410896913, 457.3204026367601), (1468.539381629429, 486.4395339304793), 'The Tutte polynomial of a graph G = (V, E) (see [11,13]) is the two-variable polynomial', 'basic']
[2, (453.49745216120874, 529.0068136461069), (1475.2628015536473, 603.5273385790792), 'T (G; x, y) = A⊆E (x − 1) κ(V,A)−κ(V,E) (y − 1) |A|−n+κ(V,A) ,(1)

[4, (200.03563410896913, 1712.1104496065693), (1475.2628015536473, 1786.2697639414323), 'In particular, under the assumption RP ̸ = NP, Corollary 9 and Lemma 13 show that there is no FPRAS for PlanarTutte(x, y) in the following cases:', 'basic']
[4, (239.79271638812673, 1823.1132449485617), (615.5263157894738, 2013.3323029989451), '1. x < 0, y < 0 and q > 5; 2. x < 1, y < 1 and q = 3; 3. x > 1, y < −1; The lower branch of the q = 3 hyperbola (also depicted in gray) is shown to be intractable in Lemma 13.', 'basic']
[5, (200.03563410896913, 1055.3740113829551), (1475.290584280607, 1270.4888152722435), 'As Vertigan has shown [12], it is easy to compute the polynomial exactly on the hyperbolas q = 1 and q = 2 and at the two special points (1, 1) and (−1, −1).(These are shown in green.) Figure2: The complexity of the planar case.The shaded gray regions are shown to be intractable in Corollary 9.The lower branch of the q = 3 hyperbola (also depicted in gray) is shown to be intractable in Le

[6, (200.03563410896913, 1759.623535973229), (1475.290584280607, 1934.6161780417822), 'Let W be a set of edge weights (for example, W might contain the edge weights α 1 , α 2 and α 3 from above) and fix a value q.Let w * be a weight (which may not be in W ) which we want to "implement".Suppose that there is a planar graph Υ , with distinguished vertices s and t on the outer face, and a weight function w : E(Υ ) → W such that', 'basic']
[6, (682.5104704890188, 1947.2307635449774), (1475.2628015536473, 1985.546872375167), 'w * = qZ st (Υ )/Z s|t (Υ ),(3)', 'basic']
[7, (200.03563410896913, 313.0028759768578), (1475.2628015536473, 542.8161726661245), 'where Z st (Υ ) denotes the contribution to Z(Υ ; q, w) arising from edge-sets A in which s and t are in the same component.That is, Z st (Υ ) = A w(A)q κ(V,A) , where the sum is over subsets A ⊆ E(Υ ) in which s and t are in the same component.Similarly, Z s|t denotes the contribution to Z(Υ ; q, w) arising from edge-sets A in which s and t

[9, (200.03563410896913, 1422.3361936311894), (1475.2628015536473, 1491.605272176268), 'Given a positive constant ρ which is sufficiently small with respect to q, α 1 , α 2 , and α 3 , there is a planar graph Υ (depending on ρ) and a weight function w : E(Υ ) → {α 1 , α 2 , α 3 } that implements a weight b, such that B − ≤ |b| ≤ B + , and(9)', 'uri:extthm.lemma.2']
[9, (696.0128757913743, 1621.3632331893104), (1460.815783534666, 1678.9902163030258), '− ρ ≤ b + c ≤ ρ. (10', 'uri:extthm.lemma.2']
[9, (1460.815783534666, 1624.419630557926), (1475.2628015536473, 1653.5387618516454), ')', 'uri:extthm.lemma.2']
[9, (200.03563410896913, 1693.2997130742713), (912.8292769839292, 1726.892298698419), 'The size of Υ is at most a polynomial in log(ρ −1 ).', 'uri:extthm.lemma.2']
[9, (200.03563410896913, 1762.541006188726), (1475.2628015536473, 1915.138591174514), 'Lemma 3. Suppose q / ∈ [0, 5] and that α 2 ∈ (−2, 0).Given a positive constant ϱ which is sufficiently small with respect to q and α 2 ,

...

[14, (200.03563410896913, 998.1915951592202), (1475.290584280607, 1153.2342980399008), 'Since G is the 3-stretch of a cubic planar graph, we will have m = 9  8 n.We will also assume that K ≤ 5  8 n, since this is an easy upper bound on the size of any independent set in G.We will rely on the following inequalities, which follow from these considerations as long as n is sufficiently large.', 'basic']
[14, (703.6253429782988, 1194.4123062243395), (1475.290584280607, 1477.8792694481215), 'L ≤ 1 (16) R ≥ 1 (17) ν ≥ 5 8 n ≥ 1 (18) 0 < δ < ε < χ ≤ 1 (19) δ ≤ εη/(6A * ) (20)', 'basic']
[14, (200.03563410896913, 1581.2132859379517), (1475.2628015536473, 1716.0559807551456), 'Suppose that quantity a satisfies ( 5), ( 6) and ( 7) with ϱ = ε and that b satisfies ( 9) and ( 10) with ρ = δ.Define c, d and e via equations ( 8), ( 12) and ( 13) respectively.Note that d ∈ [−δ, δ] and e ∈ [ε, 2ε].Note also that (11) implies', 'basic']
[14, (751.4116333487748, 1731.7825344882037), (1475.2628015536473, 1

[17, (200.03563410896913, 868.5725612992966), (1475.2628015536473, 963.2930942957553), 'For a labelled partition Π of V (G) as above, let Z Π be the contribution to Z( G; q, w) from edge sets A ∈ A Π , specifically', 'basic']
[17, (578.1307653018804, 980.3255632681311), (1460.815783534666, 1055.0405862154698), 'Z Π = A∈A Π B⊆E ′ w(A)w(B)q κ( V ,A∪B) . (25', 'basic']
[17, (1460.815783534666, 986.4661434359859), (1475.2628015536473, 1015.5852747297052), ')', 'basic']
[17, (200.03563410896913, 1101.2477573428123), (1475.2628015536473, 1257.401877448444), 'and let Γ denote the graph Γ = (V ′ , E ′ ).Suppose Π = {S, D 0 , D 1 , D 2 , T ) is some labelled partition, and denote by Γ Π the graph obtained from Γ by identifying certain vertices.Specifically, ⟨x, 0⟩, ⟨x, 1⟩ and ⟨x, 2⟩ are identified if x ∈ S, ⟨x, 1⟩ and ⟨x, 2⟩ are identified if x ∈ D 0 (and symmetrically for D 1 and D 2 ), and none of the vertices are identified if x ∈ T . It is clear that Z( G; q, w) = Π Z Π , where Π ranges ove

[19, (200.03563410896913, 1671.2936520202393), (403.4885436339665, 1700.4127833139587), 'Thus, by (28),', 'basic']
[19, (548.847771086484, 1704.1916018787924), (1125.0615282281535, 1786.1030513576898), '|Z Π | ≤ q 2 Z 012 Z 0|1|2 k |Z 0|1|2 | n |q| −3n 2 2m Q 3n−m', 'basic']
[19, (200.03563410896913, 1810.3319468616237), (507.20146337407505, 1839.451078155343), 'and by ( 24) and (17),', 'basic']
[19, (357.1469550653887, 1877.2948346649284), (1460.815783534666, 2003.3573334050088), 'Π:D=∅, |S|<K, S is independent |Z Π | ≤ 2 n • q 2 Z 012 Z 0|1|2 K−1 |Z 0|1|2 | n |q| −3n 2 2m Q 3n−m ≤ Ψ/16. (33', 'basic']
[19, (1460.815783534666, 1909.776003064125), (1475.2628015536473, 1938.8951343578442), ')', 'basic']
[20, (200.03563410896913, 313.0028759768578), (1475.2628015536473, 422.3941163426712), 'To see that the final inequality in (33) holds, plug in the definition of Ψ, and, inside that, plug in the definition of R, and inside that, plug in the definition of ε.Everything cancels exactly.', '

[22, (692.7622967371035, 986.9107103259664), (1063.828398009241, 1086.7437625572004), 'y 2 = q x j − 1 + 1 ∈ (−1, 1).', 'basic']
[22, (302.16493841238173, 1192.5228969419227), (1027.2941120573948, 1227.5881103891304), 'The deferred proofs from Section 2.3', 'basic']
[22, (200.03563410896913, 1258.7911489796331), (1475.2628015536473, 1393.633843796827), 'In this section we provide the proofs of Lemmas 1, 2 and 3. We start with some technical lemmas which show that we can use α 1 , α 2 and α 3 to implement a very close approximation to any target weight T * , provided T * / ∈ [−2, 0].', 'basic']
[22, (200.03563410896913, 1399.9411365484245), (1475.2628015536473, 1672.960777857666), 'Lemma 10.Suppose q > 5 and that α 1 / ∈ [−2, 0], α 2 ∈ (−2, 0) and α 3 < −1.Suppose that T − and T + satisfy 0 < T − ≤ T + .Given a target edge-weight T * ∈ [T − , T + ] and a positive value π which is sufficiently small with respect to q, α 1 , α 2 , α 3 , T − , and T + , there is a planar graph Υ (depending

[24, (200.03563410896913, 1954.1771212009219), (1475.2628015536473, 2027.9474395070522), 'and that the lower bound (T − − 1)/ŷ − 1 is positive.Using Lemma 10 or 11 (whichever is appropriate, depending on the sign of q) with target edge weight U * and error value π/ŷ, implement an edge-weight whose y-coordinate y ′ satisfies', 'uri:extthm.proof.9']
[25, (200.03563410896913, 308.52942164642957), (1010.5411277007687, 342.12200727057706), '', 'uri:extthm.proof.9']
[25, (690.2618513107413, 368.3514537794233), (988.342728860065, 448.4568502677749), '−T ŷ − π ŷ ≤ y ′ ≤ −T ŷ .', 'uri:extthm.proof.9']
[25, (200.03563410896913, 472.24117888172884), (1475.2628015536473, 571.4351662086158), 'Then take y ′ in parallel with y ′ 3 and j copies of y ′ 2 to get a value y = −y ′ ŷ satisfying T ≤ y ≤ T + π.', 'basic']
[25, (200.03563410896913, 579.9930788407395), (1475.290584280607, 649.2343719551939), 'We can now provide the proof of Lemmas 1, 2 and 3.For convenience, we re-state these lemmas here.', 'b

In [3]:
# 5 filtering the outputs generated using the annot_file
from pdf_is_valid import pdf_is_valid

destination = "/Users/mv96/Downloads/ACM_Multimedia/dest-pdfs"
pos, neg = pdf_is_valid(destination, n_jobs=4).fit()

print(len(pos), len(neg))
# 2665 858

with open("negative_pdfs.txt", "w") as f:
    for sample in neg:
        f.write(sample + "\n")

f.close()


100%|█████████████████████████████████████████| 379/379 [00:04<00:00, 94.30it/s]


379 0


In [9]:
# we can remove the remaining ones
from shutil import rmtree

for files in neg:
    rmtree(files.rsplit("/", 1)[0])


In [4]:
sets = []
for pdf_file in pos:
    grobid_xml = pdf_file.replace(".pdf", ".tei.xml")
    labels_xml = pdf_file.replace(".pdf", "_annot.xml")
    pdf_alto_xml_main = pdf_file.replace(".pdf", ".xml")
    sample_pdf = pdf_file
    sets.append((grobid_xml, labels_xml, pdf_alto_xml_main, sample_pdf))

print(len(sets))


379


In [5]:
from dataframe_for_eval import preprocessed_dataframe

excel_file = "output.xlsx"  # the path xlsx of the manually labelled fonts


prep = preprocessed_dataframe(sets=sets, excel_file=excel_file, n_jobs=-2)

prep.fit()


  4%|█▌                                        | 14/379 [00:01<00:47,  7.66it/s][Parallel(n_jobs=-2)]: Done   2 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-2)]: Done   3 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-2)]: Done   5 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-2)]: Done   6 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-2)]: Done   7 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-2)]: Done   8 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-2)]: Done   9 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-2)]: Done  10 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-2)]: Done  11 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-2)]: Done  12 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-2)]: Done  13 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-2)]: Done  14 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-2)]: Done  15 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-2)]: Done  16 tasks      | elapsed

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,


an error occured !/Users/mv96/Downloads/ACM_Multimedia/dest-pdfs/1204.1111/MM-rect-ArXiv-v2.tei.xml


# gunzipping from source directory

## extract all gzip from the folder


In [ ]:
# gunzip to
from sh import gunzip
import os
from joblib import Parallel, delayed
from tqdm import tqdm


def unzip_file(file_location):
    try:
        gunzip(file_location)
    except:
        return file_location


def gunzip_from_drive(gunzip_dir, n_jobs=4):
    """returns the gzip file locations"""
    gz_paths = gunzip_dir
    all_files = []
    for path in os.listdir(gz_paths):
        sub_zip_path = os.path.join(gz_paths, path)
        files = os.listdir(sub_zip_path)
        files = list(map(lambda x: os.path.join(sub_zip_path, x), files))
        all_files += files

    gunzip_files = list(filter(lambda x: x.endswith(".gz"), all_files))
    unsuccessful_files = Parallel(n_jobs=n_jobs)(
        delayed(unzip_file)(gz) for gz in tqdm(gunzip_files)
    )
    return unsuccessful_files


# this function wont return anything but it will unzip the files in the destination directory
unsuccessful_zips = gunzip_from_drive(
    gunzip_dir="/Volumes/MV96 /aws_data_pos/src-pos", n_jobs=4
)
print(len(unsuccessful_zips))

In [ ]:
gzs = list(filter(lambda x: x.endswith(".gz"), os.listdir(gz_01_path)))
print(len(gzs))
